In [1]:
#For this it should be pretty simple and quick as I just need to adjust some variables and export my training and testing 
#subsets for modeling. First I have to run my imports (I won't use featuretools as I don't know how to apply it when I have
#no temporal data)
import pandas as pd

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('spotify_data_cleaned.csv')
df.head()

,Unnamed: 0,song_name,song_popularity,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence,Is_top_track
0,0,Boulevard of Broken Dreams,73,262333,0.005520,0.496,0.682,0.000029,8,0.0589,-4.095,1,0.0294,167.060,4,0.474,False
1,1,In The End,66,216933,0.010300,0.542,0.853,0.000000,3,0.1080,-6.407,0,0.0498,105.256,4,0.370,False
2,2,Seven Nation Army,76,231733,0.008170,0.737,0.463,0.447000,0,0.2550,-7.828,1,0.0792,123.881,4,0.324,False
3,3,By The Way,74,216933,0.026400,0.451,0.970,0.003550,0,0.1020,-4.938,1,0.1070,122.444,4,0.198,False
4,4,How You Remind Me,56,223826,0.000954,0.447,0.766,0.000000,10,0.1130,-5.065,1,0.0313,172.011,4,0.574,False


In [4]:
#As I was unthoughtful and did not export the transformations I did in the EDA stage I will quickily make those adjustments 
#here.
cols_to_drop = ['Unnamed: 0', 'song_name', 'Is_top_track', 'audio_mode', 'time_signature']
df.drop(columns = cols_to_drop, axis = 1,inplace = True)
df.head()

,song_popularity,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,audio_valence
0,73,262333,0.005520,0.496,0.682,0.000029,8,0.0589,-4.095,0.0294,167.060,0.474
1,66,216933,0.010300,0.542,0.853,0.000000,3,0.1080,-6.407,0.0498,105.256,0.370
2,76,231733,0.008170,0.737,0.463,0.447000,0,0.2550,-7.828,0.0792,123.881,0.324
3,74,216933,0.026400,0.451,0.970,0.003550,0,0.1020,-4.938,0.1070,122.444,0.198
4,56,223826,0.000954,0.447,0.766,0.000000,10,0.1130,-5.065,0.0313,172.011,0.574


In [5]:
#Okay, now that I have the categories that I decided would work best in the EDA seperated, its time to make the necessary 
#transformations. Thankfully this shouldn't be too hard  as I only have one row of categorical variables (key) and the rest 
#just need to be standardized so as to not have too much of an undue influence on the modeling that will be done later.
dummy = pd.get_dummies(df['key'], drop_first = True, prefix = 'key')
df = pd.concat([df, dummy], axis = 1)
#And now I need to drop the original key column
df.drop(columns = 'key', axis = 1, inplace = True)
df.head()

,song_popularity,song_duration_ms,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,...,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
0,73,262333,0.005520,0.496,0.682,0.000029,0.0589,-4.095,0.0294,167.060,...,0,0,0,0,0,0,1,0,0,0
1,66,216933,0.010300,0.542,0.853,0.000000,0.1080,-6.407,0.0498,105.256,...,0,1,0,0,0,0,0,0,0,0
2,76,231733,0.008170,0.737,0.463,0.447000,0.2550,-7.828,0.0792,123.881,...,0,0,0,0,0,0,0,0,0,0
3,74,216933,0.026400,0.451,0.970,0.003550,0.1020,-4.938,0.1070,122.444,...,0,0,0,0,0,0,0,0,0,0
4,56,223826,0.000954,0.447,0.766,0.000000,0.1130,-5.065,0.0313,172.011,...,0,0,0,0,0,0,0,0,1,0


In [6]:
df.columns

Index(['song_popularity', 'song_duration_ms', 'acousticness', 'danceability',
       'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness',
       'tempo', 'audio_valence', 'key_1', 'key_2', 'key_3', 'key_4', 'key_5',
       'key_6', 'key_7', 'key_8', 'key_9', 'key_10', 'key_11'],
      dtype='object')

In [7]:
#At this point, I have to seperate out into a train and test split so that when I standardize later I won't have the test
#info affecting the modeling. 
X = df.drop(columns = 'song_popularity', axis = 1)
y = df[['song_popularity']]
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size = 0.25, random_state = 42)

In [8]:
#Okay, now that I have the dummy variables for the key made up and things into train and test sets, I just have to apply the
#scaler to the training data and testing data and save that for later modeling (though not on the y values as it wouldn't 
#affect the models performance and would hurt readability).
cols_to_s = ['song_duration_ms', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'audio_valence']
non_scaled = ['key_1', 'key_2', 'key_3', 'key_4', 'key_5', 'key_6', 'key_7', 'key_8', 'key_9', 'key_10', 'key_11']

scaler = StandardScaler()

#I found that if you don't reset the index before you try to do this it caused problems, so I am going to do that for all my
#training and test info so that it doesn't become a problem later on.
X_tr = X_tr.reset_index()
X_te = X_te.reset_index()
y_tr = y_tr.reset_index()
y_te = y_te.reset_index()

scaled_X_tr = pd.DataFrame(scaler.fit_transform(X_tr[cols_to_s]), columns = cols_to_s)
fin_X_tr = pd.concat([scaled_X_tr, X_tr[non_scaled]], axis = 1)

scaled_X_te = pd.DataFrame(scaler.fit_transform(X_te[cols_to_s]), columns = cols_to_s)
fin_X_te = pd.concat([scaled_X_te, X_te[non_scaled]], axis = 1)

In [9]:
#Now I just want to do a quick sanity check before I export my training and testing data for modeling
fin_X_tr.head()

,song_duration_ms,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,audio_valence,...,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
0,-0.867944,-0.924840,0.068885,1.463252,-0.388402,-0.930734,0.653316,-0.130142,0.989350,1.175261,...,0,0,0,0,0,0,0,1,0,0
1,-0.758651,-0.182371,0.778098,-0.379999,-0.392026,0.957057,0.655993,-0.338317,0.366029,-0.505265,...,0,0,0,0,0,0,0,1,0,0
2,-0.035969,-0.804983,0.577259,0.258221,1.430475,-0.377046,-0.103703,-0.447093,-0.802184,0.901874,...,0,0,0,0,0,0,0,0,0,0
3,0.530360,1.842115,-1.688456,-1.495769,-0.355902,-0.735111,-1.080942,-0.637451,-1.397357,-0.971631,...,1,0,0,0,0,0,0,0,0,0
4,-0.594471,-0.852443,-0.050363,0.637583,-0.308852,1.129869,0.664753,0.299336,1.808645,-0.686183,...,0,0,0,0,0,0,0,0,0,0


In [10]:
fin_X_te.head()

,song_duration_ms,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,audio_valence,...,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
0,0.690136,0.923233,0.504953,-0.024704,-0.391987,-0.381865,0.304084,-0.720227,-0.632693,-0.264326,...,0,0,0,0,0,0,0,1,0,0
1,-0.834349,-0.538294,1.525632,1.043371,0.663996,-0.642017,0.057160,-0.577136,-1.000947,1.800136,...,0,0,0,0,0,0,0,0,1,0
2,-0.319813,1.263047,-0.896103,-1.231316,-0.391987,2.049838,-0.553830,0.693788,-0.192700,-1.304621,...,0,0,0,0,0,0,0,0,0,1
3,-0.325001,-0.822022,0.783896,-0.190054,-0.391938,-0.963980,-1.998916,-0.506584,-0.235281,1.493693,...,0,0,0,0,0,0,0,0,0,0
4,0.698704,-0.902125,-0.984858,1.418761,1.311985,2.110970,0.417095,-0.506584,1.692083,0.634844,...,0,0,0,0,0,0,0,1,0,0


In [11]:
y_tr.head()

,index,song_popularity
0,3369,0
1,11459,57
2,10251,36
3,8784,49
4,9514,52


In [12]:
y_te.head()

,index,song_popularity
0,4695,66
1,4814,51
2,9676,51
3,5836,1
4,5405,66


In [13]:
#Well it looks like my hasty judgement to reindex all rows did not go 100% unpunished, so I'm going to quickly delete the 
#extra index row from the y test and y train before I export this out for modeling.
X_tr = fin_X_tr
X_te = fin_X_te
y_tr.drop(columns = 'index', axis = 1, inplace = True)
y_te.drop(columns = 'index', axis = 1, inplace = True)

In [14]:
y_tr.head()

,song_popularity
0,0
1,57
2,36
3,49
4,52


In [15]:
y_te.head()

,song_popularity
0,66
1,51
2,51
3,1
4,66


In [16]:
#Okay, now everything looks ready for modeling. I'll go ahead and export these out!
X_tr.to_csv('X_train.csv', index = False)
X_te.to_csv('X_test.csv', index = False)
y_tr.to_csv('y_train.csv', index = False)
y_te.to_csv('y_test.csv', index = False)